In [1]:
!pip install mediapipe opencv-python

  Using cached protobuf-4.25.6-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached sentencepiece-0.2.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 8.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 5.9 MB/s eta 0:00:0000:0100:01
Using cached protobuf-4.25.6-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 8.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 MB 11.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 MB 6.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.4/670.4 kB 7.0 MB/s eta 0:00:00
Using cached opt_einsum-3.4.0-py3-none-any.whl (71 kB)
  Attempting uninstall

In [4]:
import mediapipe as mp
import cv2
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_pushup(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    # Variables to track pushup state
    pushup_count = 0
    pushup_stage = None  # "up" or "down"
    
    # Lists to store angles for analysis
    elbow_angles = []
    body_alignment_scores = []
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        frame_count += 1
        
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Get key points for pushup analysis
            # Shoulders, elbows, wrists
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate elbow angle (important for pushup depth)
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Get right elbow angle (shoulder-elbow-wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            elbow_angles.append(right_elbow_angle)
            
            # Check body alignment (straight back)
            # Get hip, shoulder, and ankle points
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Calculate body alignment score (how straight the body is)
            # Higher score means better alignment
            hip_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            hip_ankle_angle = calculate_angle(right_shoulder, right_hip, right_ankle)
            
            # Ideally both angles should be close to 180 degrees in a proper pushup
            alignment_score = (180 - abs(hip_shoulder_angle - 180) - abs(hip_ankle_angle - 180))/180
            body_alignment_scores.append(alignment_score)
            
            # Determine pushup stage based on elbow angle
            if right_elbow_angle > 150:
                pushup_stage = "up"
            
            # If we were in "up" position and now angle is < 90, we're in "down" position
            elif right_elbow_angle < 90 and pushup_stage == "up":
                pushup_stage = "down"
                pushup_count += 1
                
    cap.release()
    
    # Analyze the data collected
    avg_elbow_angle_bottom = min(elbow_angles)
    avg_elbow_angle_top = max(elbow_angles)
    avg_alignment = sum(body_alignment_scores) / len(body_alignment_scores)
    
    # Generate feedback
    feedback = {
        "pushup_count": pushup_count,
        "form_analysis": {
            "elbow_angle_at_bottom": avg_elbow_angle_bottom,
            "elbow_angle_at_top": avg_elbow_angle_top,
            "body_alignment_score": avg_alignment * 100  # Convert to percentage
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if avg_elbow_angle_bottom > 90:
        feedback["feedback"].append("You're not going deep enough. Try to lower your body until your elbows are at 90 degrees.")
    
    if avg_elbow_angle_top < 150:
        feedback["feedback"].append("You're not fully extending your arms at the top of the pushup.")
    
    if avg_alignment < 0.8:
        feedback["feedback"].append("Keep your body straighter throughout the movement. Your hips are sagging or piking.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your pushups have good depth and body alignment.")
        
    return feedback

# Example usage
result = analyze_pushup("pushup.MOV")
print(f"Counted {result['pushup_count']} pushups")
print(f"Body alignment score: {result['form_analysis']['body_alignment_score']:.1f}%")
print("\nFeedback:")
for item in result["feedback"]:
    print(f"- {item}")

I0000 00:00:1743722769.996126 14958205 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4
W0000 00:00:1743722770.078483 14967007 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743722770.089992 14967009 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Counted 0 pushups
Body alignment score: 10.5%

Feedback:
- You're not fully extending your arms at the top of the pushup.
- Keep your body straighter throughout the movement. Your hips are sagging or piking.


In [5]:
import mediapipe as mp
import cv2
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_pushup(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track pushup state
    pushup_count = 0
    pushup_stage = None  # "up" or "down"
    frames_without_detection = 0
    good_frames = 0
    
    # Lists to store angles for analysis
    elbow_angles = []
    body_alignment_scores = []
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            frames_without_detection = 0
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get key points for pushup analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Get right elbow angle (shoulder-elbow-wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            elbow_angles.append(right_elbow_angle)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize elbow angle
            r_shoulder_px = normalized_to_pixel_coordinates(right_shoulder[0], right_shoulder[1], frame_width, frame_height)
            r_elbow_px = normalized_to_pixel_coordinates(right_elbow[0], right_elbow[1], frame_width, frame_height)
            r_wrist_px = normalized_to_pixel_coordinates(right_wrist[0], right_wrist[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"Elbow: {right_elbow_angle:.1f}°",
                        (r_elbow_px[0] - 50, r_elbow_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Body alignment check
            # Get hip, shoulder, and ankle points
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Calculate body alignment
            hip_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            hip_ankle_angle = calculate_angle(right_shoulder, right_hip, right_ankle)
            
            # Better alignment calculation
            # Perfect alignment would have both angles close to 180
            alignment_score = (180 - abs(hip_shoulder_angle - 180) - abs(hip_ankle_angle - 180))/180
            body_alignment_scores.append(alignment_score)
            
            # Visualize body alignment
            cv2.putText(annotated_image, 
                        f"Alignment: {alignment_score*100:.1f}%",
                        (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
            
            # Determine pushup stage based on elbow angle
            if right_elbow_angle > 150 and pushup_stage == "down":
                pushup_stage = "up"
                # Draw pushup count
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # If we were in "up" position and now angle is < 90, we're in "down" position
            elif right_elbow_angle < 100 and (pushup_stage == "up" or pushup_stage is None):
                pushup_stage = "down"
                pushup_count += 1
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                print(f"Pushup #{pushup_count} detected at frame with elbow angle {right_elbow_angle:.1f}")
        else:
            frames_without_detection += 1
            if frames_without_detection % 30 == 0:
                print(f"No pose detection for {frames_without_detection} frames")
                
        # Display pushup count
        cv2.putText(annotated_image, f'Pushups: {pushup_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "pushup_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Analyze the data collected
    if elbow_angles:
        avg_elbow_angle_bottom = min(elbow_angles)
        avg_elbow_angle_top = max(elbow_angles)
    else:
        avg_elbow_angle_bottom = 0
        avg_elbow_angle_top = 0
        
    avg_alignment = sum(body_alignment_scores) / max(len(body_alignment_scores), 1)
    
    # Generate feedback
    feedback = {
        "pushup_count": pushup_count,
        "form_analysis": {
            "elbow_angle_at_bottom": avg_elbow_angle_bottom,
            "elbow_angle_at_top": avg_elbow_angle_top,
            "body_alignment_score": avg_alignment * 100,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if avg_elbow_angle_bottom > 100:
        feedback["feedback"].append("You're not going deep enough. Try to lower your body until your elbows are at 90 degrees.")
    
    if avg_elbow_angle_top < 150:
        feedback["feedback"].append("You're not fully extending your arms at the top of the pushup.")
    
    if avg_alignment < 0.8:
        feedback["feedback"].append("Keep your body straighter throughout the movement. Your hips are sagging or piking.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your pushups have good depth and body alignment.")
        
    return feedback

# Example usage
result = analyze_pushup("pushup.MOV", "analyzed_pushup.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('pushup_count', 0)} pushups")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Body alignment score: {result['form_analysis']['body_alignment_score']:.1f}%")
    print(f"Lowest elbow angle: {result['form_analysis']['elbow_angle_at_bottom']:.1f}°")
    print(f"Highest elbow angle: {result['form_analysis']['elbow_angle_at_top']:.1f}°")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1743722914.783497 14958205 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4
W0000 00:00:1743722914.855040 14971231 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743722914.866219 14971231 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29
Detected 33 landmarks
Pushup #1 detected at frame with elbow angle 97.1

=== ANALYSIS RESULTS ===
Counted 1 pushups
Body alignment score: 10.5%
Lowest elbow angle: 6.9°
Highest elbow angle: 126.2°
Frames analyzed: 86

Feedback:
- You're not fully extending your arms at the top of the pushup.
- Keep your body straighter throughout the movement. Your hips are sagging or piking.


In [6]:
import mediapipe as mp
import cv2
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def analyze_bicep_curl(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track bicep curl state
    curl_count = 0
    curl_stage = None  # "up" or "down"
    good_frames = 0
    
    # Lists to store angles and positions for analysis
    elbow_angles = []
    shoulder_stability = []
    hip_stability = []
    wrist_angles = []
    rep_depths = []  # To track curl depth
    
    # Add debugging info
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Debug output to see what landmarks are detected
            if good_frames == 1:
                print(f"Detected {len(landmarks)} landmarks")
            
            # Get key points for curl analysis
            # We'll focus on right arm for simplicity, but you could analyze both
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Get elbow angle (shoulder-elbow-wrist)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)
            elbow_angles.append(elbow_angle)
            
            # Track shoulder stability - shoulder should remain still relative to hip
            shoulder_hip_distance = np.sqrt((shoulder[0] - hip[0])**2 + (shoulder[1] - hip[1])**2)
            shoulder_stability.append(shoulder_hip_distance)
            
            # Measure wrist angle relative to forearm (to detect wrist curling/improper form)
            # For simplicity, we'll estimate this as deviation from straight line
            wrist_deviation = abs(elbow_angle - 180) if elbow_angle > 90 else elbow_angle
            wrist_angles.append(wrist_deviation)
            
            # Convert normalized coordinates to pixel coordinates for visualization
            def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
                x_px = min(math.floor(normalized_x * image_width), image_width - 1)
                y_px = min(math.floor(normalized_y * image_height), image_height - 1)
                return x_px, y_px
            
            # Visualize elbow angle
            shoulder_px = normalized_to_pixel_coordinates(shoulder[0], shoulder[1], frame_width, frame_height)
            elbow_px = normalized_to_pixel_coordinates(elbow[0], elbow[1], frame_width, frame_height)
            wrist_px = normalized_to_pixel_coordinates(wrist[0], wrist[1], frame_width, frame_height)
            
            # Draw angle text
            cv2.putText(annotated_image, 
                        f"Elbow: {elbow_angle:.1f}°",
                        (elbow_px[0] - 50, elbow_px[1] + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine curl stage based on elbow angle
            if elbow_angle > 160 and curl_stage == "up":
                curl_stage = "down"
                # Draw curl stage
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # If we were in "down" position and now angle is < 60, we're in "up" position
            elif elbow_angle < 60 and (curl_stage == "down" or curl_stage is None):
                curl_stage = "up"
                curl_count += 1
                rep_depths.append(elbow_angle)  # Store depth of curl
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                print(f"Curl #{curl_count} detected at frame with elbow angle {elbow_angle:.1f}")
                
        # Display curl count
        cv2.putText(annotated_image, f'Curls: {curl_count}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "curl_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
        
    # Calculate metrics for analysis
    avg_shoulder_stability = np.std(shoulder_stability) * 100  # Convert to percentage variation
    avg_wrist_deviation = np.mean(wrist_angles)
    avg_curl_depth = np.mean(rep_depths) if rep_depths else 0
    full_extension = np.max(elbow_angles) > 150
    
    # Generate feedback
    feedback = {
        "curl_count": curl_count,
        "form_analysis": {
            "curl_depth": avg_curl_depth,
            "full_extension": full_extension,
            "shoulder_stability": 100 - avg_shoulder_stability,  # Higher is better
            "wrist_stability": 100 - avg_wrist_deviation,  # Higher is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if avg_curl_depth > 45:
        feedback["feedback"].append("You're not curling the weight fully to your shoulder. Try to bring the weight closer to your shoulder.")
    
    if not full_extension:
        feedback["feedback"].append("You're not fully extending your arm at the bottom of the curl. Straighten your arm more for full range of motion.")
    
    if avg_shoulder_stability > 10:  # If shoulder position varies too much
        feedback["feedback"].append("Keep your upper arm and shoulder more stable during the curl. Avoid swinging.")
    
    if avg_wrist_deviation > 30:
        feedback["feedback"].append("Keep your wrist straight throughout the movement. Avoid bending your wrist.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your bicep curls show good depth, stable shoulders, and proper wrist position.")
        
    return feedback

# Example usage
result = analyze_bicep_curl("curl.MOV", "analyzed_curl.mp4")
print("\n=== ANALYSIS RESULTS ===")
print(f"Counted {result.get('curl_count', 0)} curls")
if "error" in result:
    print(f"Error: {result['error']}")
elif "form_analysis" in result:
    print(f"Curl depth (lower is better): {result['form_analysis']['curl_depth']:.1f}°")
    print(f"Full arm extension: {'Yes' if result['form_analysis']['full_extension'] else 'No'}")
    print(f"Shoulder stability: {result['form_analysis']['shoulder_stability']:.1f}%")
    print(f"Wrist stability: {result['form_analysis']['wrist_stability']:.1f}%")
    print(f"Frames analyzed: {result['form_analysis']['frames_analyzed']}")
    print("\nFeedback:")
    for item in result["feedback"]:
        print(f"- {item}")

I0000 00:00:1743723293.591700 14958205 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4
W0000 00:00:1743723293.665656 14980167 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743723293.679111 14980173 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video dimensions: 1920x1080, FPS: 29
Detected 33 landmarks

=== ANALYSIS RESULTS ===
Counted 0 curls
Curl depth (lower is better): 0.0°
Full arm extension: Yes
Shoulder stability: 99.0%
Wrist stability: 36.9%
Frames analyzed: 188

Feedback:
- Keep your wrist straight throughout the movement. Avoid bending your wrist.
